In [1]:
from tensorflow.keras.utils import to_categorical
import segmentation_models as sm
import matplotlib.pyplot as plt
import numpy as np
import sys, os
import tensorflow as tf
from tensorflow.keras.utils import normalize
from tensorflow.keras.metrics import MeanIoU, IoU
from tensorflow.keras.optimizers import Adam

Segmentation Models: using `keras` framework.


In [2]:
sys.path.insert(1, 'E:/Research/course research projects/data-driven software engineering/gui-element-detection/src')
from constants import VINS_MERGED_YOLO_SPLITS_IMAGES as imgs_path
from constants import VINS_MERGED_YOLO_SPLITS_LABELS as labels_path
from util.data_utils import load_data, load_labels_as_masks, Splits
# from ..constants import VINS_MERGED_YOLO_SPLITS_IMAGES as imgs_path
# from ..constants import VINS_MERGED_YOLO_SPLITS_LABELS as labels_path

In [3]:
train_x = load_data(Splits.TRAIN, path=imgs_path, resize=True, shape=(416, 416), normalize=True)

val_x = load_data(Splits.VALIDATION, path=imgs_path, resize=True, shape=(416, 416), normalize=True)


Split TRAIN data was loaded with 3640 images
Split VALIDATION data was loaded with 455 images


In [4]:
train_y_m = load_labels_as_masks(Splits.TRAIN, path=labels_path, shape=(416, 416))
val_y_m = load_labels_as_masks(Splits.VALIDATION, path=labels_path, shape=(416, 416))

E:/Research/course research projects/data-driven software engineering/datasets/All Dataset/Merged/dataset/labels/train\10055.txt
E:/Research/course research projects/data-driven software engineering/datasets/All Dataset/Merged/dataset/labels/train\1007.txt
E:/Research/course research projects/data-driven software engineering/datasets/All Dataset/Merged/dataset/labels/train\10107.txt
E:/Research/course research projects/data-driven software engineering/datasets/All Dataset/Merged/dataset/labels/train\1014.txt
E:/Research/course research projects/data-driven software engineering/datasets/All Dataset/Merged/dataset/labels/train\1016.txt
E:/Research/course research projects/data-driven software engineering/datasets/All Dataset/Merged/dataset/labels/train\10237.txt
E:/Research/course research projects/data-driven software engineering/datasets/All Dataset/Merged/dataset/labels/train\10244.txt
E:/Research/course research projects/data-driven software engineering/datasets/All Dataset/Merged/da

In [5]:
# test_x = load_data(Splits.TEST, path=imgs_path, resize=True, shape=(416, 416), normalize=True)
# test_y_m = load_labels_as_masks(Splits.TEST, path=imgs_path, shape=(416, 416))

In [6]:
train_y = to_categorical(train_y_m, num_classes=13)
val_y = to_categorical(val_y_m, num_classes=13)
# test_y = to_categorical(test_y_m, num_classes=13)

In [7]:
activation = 'softmax'
learning_rate = 0.0001
opt = Adam(learning_rate)
dice_loss = sm.losses.DiceLoss(class_weights=np.array([0.25,0.25,0.25,0.25])) # makes sense as the end result is highly imbalanced
focal_loss = sm.losses.CategoricalFocalLoss() # why not Categorical CE ?
total_loss = dice_loss + (focal_loss * 1)

metrics = [sm.metrics.IOUScore(threshold=0.5),sm.metrics.IOUScore(threshold=0.9), sm.metrics.FScore(threshold=0.5), sm.metrics.Precision(), sm.metrics.Recall()] # this is all they have :|

In [8]:
RESNET_BACKBONE = 'resnet34'
preprocess_inputs = sm.get_preprocessing(RESNET_BACKBONE)

prep_train_x = preprocess_inputs(train_x)
prep_val_x = preprocess_inputs(val_x)
# prep_test_x =

In [9]:
train_y.shape

(3640, 416, 416, 13)

In [10]:
classes = 12 + 1
model1 = sm.Unet(RESNET_BACKBONE, encoder_weights='imagenet', classes= 13, activation=activation)
model1.compile(opt, total_loss, metrics=metrics)

In [11]:
model1.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 bn_data (BatchNormalization)   (None, None, None,   9           ['data[0][0]']                   
                                3)                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, None, None,   0          ['bn_data[0][0]']                
                                3)                                                          

In [12]:
del train_y_m
del val_y_m

In [13]:
history1=model1.fit(prep_train_x,
                    train_y,
                    batch_size=1,
                    epochs=100,
                    verbose=2,
                    validation_data=(prep_val_x, val_y))

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
model1.save('res34bb_UNET_100epochs.hdf5')